Welcome to my ambitious single do-it-all script.

In [2]:
import sys, os, subprocess
galaxy_folder_path = os.getcwd()
sys.path.insert(0, '../')
import DustpediaScripts as DS
sys.path.insert(0, '../scripts/')
import GenericUsefulScripts as GUS
from astropy import units as u
import pandas as pd

import DataReduction as DataRed
import PhotometryScripts as PhotoScripts
import SedFittingScripts as SEDfit
import ResultsScripts as ResScripts

%matplotlib inline
# Or widget, if you feel brave

# Change these lines accordingly
galaxy_name = ' '
working_bands = ['GALEX_FUV', 'GALEX_NUV', 'SDSS_u', 'SDSS_g', 'SDSS_r', 'SDSS_i', 'SDSS_z', \
                 '2MASS_J', '2MASS_H', '2MASS_Ks', 'WISE_3.4', 'Spitzer_3.6', 'Spitzer_4.5', \
                 'WISE_4.6', 'Spitzer_8.0', 'WISE_12', 'WISE_22',\
                 'PACS_70', 'PACS_100', 'PACS_160', 'SPIRE_250', 'SPIRE_350']
# Not all error maps are available, usually it reduces to these ones
# For other bands, error will come from rms evaluation
error_bands = ['Spitzer_8.0', 'PACS_70', 'PACS_100', 'PACS_160', 'SPIRE_250', 'SPIRE_350']

galaxy_properties = GUS.GalaxyProperties(galaxy_name)

- First things first, download the data, run CAAPR for starsub

In [3]:
do_download = False
do_download_errors = False

if do_download == True:
    GUS.download_data(galaxy_name, working_bands, download_directory = 'Caapr/Maps/')
    subprocess.call('mkdir _ReducedMaps', shell = True)
if do_download_errors == True:
    subprocess.call('mkdir _ReducedMaps', shell = True)
    GUS.download_data_errors(galaxy_name, error_bands, download_directory = '_ReducedMaps/ERROR_MAPS/')

- In principle, you want to use CAAPR to star subtract the images. In practice, the starsub is far from perfect, so I usually skip this part

In [3]:
#do_CAAPR = False
#if do_CAAPR == True: GUS.run_CAAPR(galaxy_name)

- Once we have the CAAPR star subtracted images, it's time for a little data reduction of our own.

In [4]:
do_datareduction = False
check_datareduction = False 

if do_datareduction: DataRed.data_reduction_parallel(galaxy_name, processes = 5, path_fits_input = 'Caapr/Maps')
if check_datareduction: DataRed.check_Dustpedia(galaxy_name, working_bands)

- Ok, next phase is photometry. This is articulated in three steps:
    - zero, evaluate rms per pixel in each band
    - first, generate the grid of coordinates
    - second, perform photometry
    - third, build the tables to feed MAGPHYS for SED fitting

In [5]:
# Difference between v1 and v2:
# -v1 is new, and I rewrote the estimation after this year in Bologna
# -v2 is the one I used in the papers, basically copy-pasted from CAAPR
# More details in the README.
evaluate_rms = False
if evaluate_rms:
    PhotoScripts.evaluate_rms(galaxy_properties, version = 'v1')

In [2]:
# Change these values accordingly
run_type = ' '
aperture_side = 
reference_map_4grid = 
reference_map_4check =  
grid_extention_from_center =  
plot_radius =  

In [6]:
generate_grid = False
check_grid = False

if generate_grid:
    # The semimajor axis, angle and axial ratio over which the script will NOT generate coordinates
    # If False, the values are taken from DustPedia tables
    semimaj_avoidance =  
    avoidance_angle = 
    avoidance_axial_ratio =  
    PhotoScripts.generate_coordinate_grid_within_DustAp(galaxy_properties, reference_map_4grid, aperture_side, \
                                                        run_type, grid_extention_from_center, \
                                                        semimaj_avoidance, avoidance_angle, avoidance_axial_ratio)
if check_grid:
    PhotoScripts.check_coordinate_grid(galaxy_properties, reference_map_4check, run_type, plot_radius)

In [7]:
do_photometry = False
if do_photometry:
    PhotoScripts.do_photometry_build_table(galaxy_properties, working_bands, aperture_side, run_type, \
                                  SNR_threshold = 2, skip_value = 10, remove_weird_points = True)

- There's not much left to do, just run MAGPHYS

In [8]:
do_magphys_preliminary = False
do_run_magphys = False
if do_magphys_preliminary:
    exclude_filters = False # If you want to exclude a filter from MAGPHYS fit, for whatever reasons
    SEDfit.prepare_magphys(galaxy_name, run_type, working_bands, 'almost_fill_cores', exclude_filters = exclude_filters)
if do_run_magphys:
    SEDfit.run_magphys(galaxy_name, run_type)

- Final phase, move magphys results out of magphys folder

In [9]:
do_move_results = True

if do_move_results:
    status = SEDfit.move_results(galaxy_name, run_type, galaxy_folder_path)
    if status == 'over': SEDfit.check_results(galaxy_name, run_type, galaxy_folder_path)

In [8]:
do_read_results = False
if do_read_results:
    ResScripts.read_magphys(galaxy_properties, run_type, aperture_side)

do_clustering_removal = False
if do_clustering_removal:
    ResScripts.clustering_removal(galaxy_properties, run_type, eps_value = , s_size = )

In [9]:
do_plot_results = False
do_plot_seds = False

if do_plot_results:
    ResScripts.plot_results(galaxy_properties, run_type, aperture_side, 'PACS_160', \
                            chisq_threshold = 25, \
                            plot_size = 15*u.kpc, s_size = 80)
if do_plot_seds:
    ResScripts.plot_seds(galaxy_properties, run_type, aperture_side, 'SDSS_u', plot_radius)

---